In [ ]:
%cd ..

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

MODEL_PATH = "unsloth/gemma-1.1-2b-it" # "unsloth/llama-3-8b-Instruct-bnb-4bit"
# MODEL_PATH = "unsloth/tinyllama-bnb-4bit"

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer
from peft import PeftModelForCausalLM

In [ ]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


result: tuple[PeftModelForCausalLM, AutoTokenizer] = FastLanguageModel.from_pretrained(
	model_name = MODEL_PATH,
	max_seq_length = max_seq_length,
	dtype = dtype,
	load_in_4bit = load_in_4bit,
	# token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
model: PeftModelForCausalLM = result[0]
tokenizer: AutoTokenizer = result[1]

In [ ]:
import json

QUERY = """
Here is the context to generate questions from:

<context>
{CONTEXT}
</context>

Please generate <num_questions>{NUM_QUESTIONS}</num_questions> questions based on the provided context.

Read the context carefully and create questions that can be clearly answered using only the information provided. Do not make questions that would require outside knowledge to answer.

For each question you create:
- Write out the question text inside "question" quotes 
- Generate at most 5 possible answer options and list them out as "option 1", "option 2", etc. Make sure the incorrect options are plausible based on the context.
- Specify the correct answer inside "answer" quotes, referring to the option number

Output each complete question with its answer options and correct answer inside <qa> tags, using this format:

<qa>
"question": "Question text goes here",
"option 1": "First answer option",
"option 2": "Second answer option", 
"option 3": "Third answer option",
"option 4": "Fourth answer option",
"option 5": "Fifth answer option",
"answer": "option X: Correct answer text",
</qa>

Generate all the questions one after another until you have created the requested number based on the context provided. Do not repeat questions.
"""

In [ ]:
NUM_QUESTIONS = 5

CONTEXT = """
The " Integration of satellite components in the 5G architecture” work item adds or enhances a number of features in 5GCore architecture in order to support Non-Terrestrial Networks (NTN), for several use cases:

Coverage extension: Many commercial activities, such as agriculture, mining, forestry take place outside inhabited areas. Coverage extension with satellite networks is useful to enable e.g. voice communication, video monitoring, and remote control in uncovered or under-covered areas. 

Internet of Things: many Internet-of-Things applications relate to monitoring of assets (e.g. ships, trains, trucks), infrastructure (e.g. bridges, pipelines, railway track), or the environment (e.g. agriculture sensors). In many IoT applications, only small amounts of data are exchanged and communication is optimized for low power usage. Satellite communication should also be able to address these requirements.

Disaster communication: Public safety authorities have a responsibility to provide assistance in case of natural disasters. This requires communication, also in cases where because of that disaster the cellular infrastructure is damaged. Satellite communication can be used as fall back for these cases. Ideally the user equipment (UE) and way of working when cellular networks are available should also be usable with satellite access.

Global roaming: Applications like tracking and tracing of containers need to be available globally across satellite and terrestrial networks. When a container is in a harbour or transported on a truck, using a terrestrial cellular network is probably most efficient. However, when the container is on a ship in the middle of an ocean, only satellite communication is possible.

Broadcasting: Satellite communication is particularly suitable to broadcast the same information over a very wide area. This can also be used in context of 5G mobile edge applications (e.g. mobile gaming), where application content needs to be available in many different edge locations.

To address such use cases, 3GPP has set Key Performance Indicator (KPI) targets for satellite in TS 22.261 [1].

At 5G Core Network architecture level, in SA2, a dedicated study on architecture aspects for using satellite access in 5G (FS_5GSAT_ARCH) was conducted to select the solutions able to cope with satellite specific key issues. The outcome of the study (TR 23.737 [2]) identifies the impacts of satellite integration in the 5GS and solutions to adjust the 5G system accordingly. 

The 5GSAT_ARCH work item, following the study, updated architecture specifications (TS 23.501 [4], TS 23.502 [5], TS 23.503 [6]) to implement the solutions identified. 

In CT1, TR 24.821 [3] studied "Non-Terrestrial Impact of PLMN selection procedure" and, following 5GSAT_ARCH_CT, led to update TS 23.122 [7] and TS 24.501 [8]. 

Furthermore, RAN has defined "3GPP defined radio access networks supporting non-Terrestrial Networks" [10], described in the next clause.

Architectural/general aspects

A PLMN core network can be connected to a satellite NG-RAN. A satellite NG-RAN can be shared between more than one core networks. 

Satellite NG-RAN can be used as a new RAN 3GPP access but also as backhaul between the core and terrestrial access network, providing a transport for the N1/N2/N3 reference points.

Multi-connectivity and URLLC over satellite are not considered in Rel-17. Basic assumptions are that UEs are equipped with GNSS, and transparent mode: satellites (LEO/MEO/GEO) are relaying the Uu interface only at physical layer level.

Impacts on 5GC of Satellite NG-RAN used as new RAN 3GPP access

In Rel-17, only direct access with transparent satellite is considered, as shown in following figure:

 

Figure 1: Direct access with transparent satellite

Impacts of satellites onto 5GS are linked to the size of the cells (larger than the terrestrial ones), the fact that the satellite cells can be fix on earth, when beam is steerable, but also moving on earth, when beam is not steerable. This characteristics impacts 5GS mobility management, i.e. the management of the handover of radio bearer between nodes and the management of the reachability of a UE for downlink services (paging), that need to be adapted to take into account both the satellite beam size and fix or moving cells configuration. 

A basic assumption in Re-17 is that tracking areas (TAs) and cell identities (cell IDs) refer to specific geographical areas, so that 5G services can use these identifiers as representation of a UE location. 

To avoid Tracking Area Codes (TAC) fluctuations, in the moving cells case, it has been decided that the Radio Access Network will broadcast in the cell the list of Tracking Area Codes, corresponding to tracking areas that have been define on the earth surface through network planning, for the zone currently enlighten by the radio cell. 

New Radio Access Technology types are introduced in the 5GC to distinguish between different satellite configurations (LEO, MEO, GEO, other).

The distance earth – satellite also introduces higher delay values than for terrestrial cells and new 5QI is also introduced in TR 23.501 [4] to cope with this delay, depending on the satellite RAT type.  

Impacts on 5GC of Satellite NG-RAN used as backhaul

Rel-17 only considers backhauling with constant delay. Here, the satellite operator is able to mask any delay changes in service/feeder links by exploiting the knowledge of the satellite position to calculate how much variable delay should be added to keep the overall delay constant. Connecting gNBs to 5GC via, e.g., a single GEO satellite or a single NGSO satellite without ISLs are examples of such backhauling as shown in following figure.
"""

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

In [ ]:
text = [
	QUERY.format(NUM_QUESTIONS=NUM_QUESTIONS, CONTEXT=CONTEXT)
]
len(text)

In [ ]:
inputs = tokenizer(
text, return_tensors = "pt", padding=True, truncation=True).to("cuda")

In [ ]:
predicted = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2096,)

In [ ]:
print(tokenizer.batch_decode(predicted)[0])